# V2：加入bend_penalty，且每个格子不是单元cost

In [97]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
from queue import Queue
filepath_out='../output/bench2.router'
netlist_file_path = '../benchmark/bench2.nl'
gridfile_path  = '../benchmark/bench2.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid_original,layer2_grid_original = parse_gridfile(gridfile_path)

bend_penalty: 50 
 via_penalty: 0
size of layer1: (40, 200)
size of layer2: (40, 200)


In [98]:
layer1_grid_original.T[31]

array([ 1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
       -1., -1., -1.,  2.,  2.,  2.,  2.,  2., -1., -1., -1.,  1.,  1.,
        1.])

In [99]:
plot_path("../output/bench2_block.jpg",columns=columns,rows=rows,block_list=np.argwhere(layer1_grid_original.T==-1),path_dict={})

In [100]:
def reconstruct_path(source, target, parents):
    path = []
    current = target
    while current != source:
        path.append(current)
        current = parents[current]
    path.append(source)
    path.reverse()
    return path

def mark_path_on_grid(layer1_grid, path):
    #print(path)
    for cell in path:
        x, y, _ = cell
        layer1_grid[x][y] = -1

In [101]:
def get_cell_cost(layer1_grid, cell,path_tmp,bend_penalty):
    x, y, layer = cell
    cell_cost = 1  # 默认的单元代价
    if layer1_grid[x][y] == -1:
        cell_cost = float('inf')  # -1表示无法通过的细胞
    elif layer1_grid[x][y] != 1:
        cell_cost = layer1_grid[x][y]  # 非单元代价
    if len(path_tmp) >= 2:
        prev_cell = path_tmp[-2]
        prev_x, prev_y, prev_layer = prev_cell
        if layer != prev_layer and (prev_x != x or prev_y != y):  # 方向变化
            cell_cost += bend_penalty
    return cell_cost

def expand_source_to_target(rows, columns, layer1_grid, source, target,bend_penalty):
    queue = Queue()
    visited = set()
    parents = {}
    costs = {}  # Store the cumulative costs for each cell
    
    source_tuple = (source['x'], source['y'], source['layer'])
    target_tuple = (target['x'], target['y'], target['layer'])
    
    queue.put(source_tuple)
    visited.add(source_tuple)
    costs[source_tuple] = 0  # Initial cost for the source cell is 0

    while not queue.empty():
        current_cell = queue.get()

        if current_cell == target_tuple:
            path = reconstruct_path(source_tuple, target_tuple, parents)
            return path,costs

        neighbors = get_neighbors(rows, columns, current_cell)

        for neighbor in neighbors:
            neighbor_tuple = (neighbor['x'], neighbor['y'], neighbor['layer'])
            path_tmp= reconstruct_path(source_tuple, current_cell, parents)
            # Calculate the cost to reach the neighbor cell
            cost = costs[current_cell] + get_cell_cost(layer1_grid, neighbor_tuple,path_tmp,bend_penalty)

            if neighbor_tuple not in visited or cost < costs[neighbor_tuple]:
                queue.put(neighbor_tuple)
                visited.add(neighbor_tuple)
                parents[neighbor_tuple] = current_cell
                costs[neighbor_tuple] = cost
    return []

def get_neighbors(rows, columns, cell):
    x, y, layer = cell
    neighbors = []
    if x > 0:
        neighbors.append({'x': x - 1, 'y': y, 'layer': layer})
    if x < columns - 1:
        neighbors.append({'x': x + 1, 'y': y, 'layer': layer})
    if y > 0:
        neighbors.append({'x': x, 'y': y - 1, 'layer': layer})
    if y < rows - 1:
        neighbors.append({'x': x, 'y': y + 1, 'layer': layer})

    return neighbors

In [102]:
def nounit_bend_penalty_cost_router(rows, columns, layer1_grid, nets,bend_penalty):
    routing_table = {}
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = -1
        layer1_grid[pin2['x']][pin2['y']] = -1
        ## 防止布线在后续的pin上，先将所有的pin标记为-1；
        
    for net in nets:
        net_id = net['net_id']
        pin1 = net['pin1']
        pin2 = net['pin2']
        layer1_grid[pin1['x']][pin1['y']] = 1
        layer1_grid[pin2['x']][pin2['y']] = 1
        path,costs = expand_source_to_target(rows, columns, layer1_grid, pin1, pin2,bend_penalty) # 少了第二个return
        mark_path_on_grid(layer1_grid,path)
        routing_table[net_id] = path
    return routing_table,costs



In [94]:
layer1_gird = layer1_grid_original
routing_table = {}
net_id = nets[0]['net_id']
pin1 = nets[0]['pin1']
pin2 = nets[0]['pin2']
layer1_gird[pin1['x']][pin1['y']] = 1
layer1_gird[pin2['x']][pin2['y']] = 1
path,cost = expand_source_to_target(rows, columns, layer1_gird.T, pin1, pin2,bend_penalty)
mark_path_on_grid(layer1_gird.T,path)
routing_table[net_id] = path

In [95]:
layer1_gird[5][31]

2.0

In [105]:
layer1_grid = layer1_grid_original
routing_table,cost1 = nounit_bend_penalty_cost_router(rows,columns,layer1_grid.T,nets,bend_penalty)
print(routing_table)
generate_output_file(filepath_out,net_num,routing_table)
plot_path('../output/bench2.jpg',columns=columns,rows=rows,block_list=np.argwhere(layer1_grid.T==-1),path_dict=routing_table)

{1: [(1, 31, 1), (2, 31, 1), (3, 31, 1), (4, 31, 1), (5, 31, 1), (6, 31, 1), (7, 31, 1), (8, 31, 1), (9, 31, 1), (10, 31, 1), (11, 31, 1), (12, 31, 1), (13, 31, 1), (14, 31, 1), (15, 31, 1), (16, 31, 1), (17, 31, 1), (18, 31, 1), (19, 31, 1)], 2: [(21, 31, 1), (22, 31, 1), (23, 31, 1), (24, 31, 1), (25, 31, 1), (26, 31, 1), (27, 31, 1), (28, 31, 1), (29, 31, 1), (30, 31, 1), (31, 31, 1), (32, 31, 1), (33, 31, 1), (34, 31, 1), (35, 31, 1), (36, 31, 1), (37, 31, 1), (38, 31, 1), (39, 31, 1)], 3: [(41, 31, 1), (42, 31, 1), (43, 31, 1), (44, 31, 1), (45, 31, 1), (46, 31, 1), (47, 31, 1), (48, 31, 1), (49, 31, 1), (50, 31, 1), (51, 31, 1), (52, 31, 1), (53, 31, 1), (54, 31, 1), (55, 31, 1), (56, 31, 1), (57, 31, 1), (58, 31, 1), (59, 31, 1)], 4: [(61, 31, 1), (62, 31, 1), (63, 31, 1), (64, 31, 1), (65, 31, 1), (66, 31, 1), (67, 31, 1), (68, 31, 1), (69, 31, 1), (70, 31, 1), (71, 31, 1), (72, 31, 1), (73, 31, 1), (74, 31, 1), (75, 31, 1), (76, 31, 1), (77, 31, 1), (78, 31, 1), (79, 31, 1)], 